# 시맨틱 검색 - Semantic Search


In [1]:
!pip install -q boto3
!pip install -q requests
!pip install -q requests-aws4auth
!pip install -q opensearch-py
!pip install -q tqdm
!pip install -q boto3

## 사전준비


### 데이터 준비 및 탐색


In [218]:
import pandas as pd
import requests

df = pd.read_csv("../data/movies.csv", low_memory=False)
df.head(5)

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,page_url,img_url
0,1917,드라마|전쟁,2020,2.19,8.88,5017,제1차 세계대전이 한창인 1917년. 독일군에 의해 모든 통신망이 파괴된 상황 속에...,조지 맥케이|딘-찰스 채프먼,콜린 퍼스|베네딕트 컴버배치|마크 스트롱|앤드류 스캇|리차드 매든|에드리언 스카보로...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200212_161/1...
1,21 브릿지: 테러 셧다운,범죄|드라마|스릴러,2020,1.01,7.84,579,"뉴욕 맨해튼 중심에서 벌어진 경찰 연쇄 살해 사건, 범인을 잡기 위해 베테랑 경찰 ...",채드윅 보스만|J.K. 시몬스|시에나 밀러|테일러 키취,스테판 제임스|빅토리아 카타게나|키스 데이빗|샤이나 라이언|알렉산더 시디그|모로코 ...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20191227_205/1...
2,n번째 이별중,코미디|멜로/로맨스,2020,4.01,6.48,177,여자친구 ‘데비’에게 대차게 차여 충격받은 물리학 천재 ‘스틸먼’. 사랑하는 그녀의...,에이사 버터필드|소피 터너|스카일러 거손도,오브리 레이놀즈|윌 펠츠|질리안 조이|케이든 J. 그레고리,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200409_56/15...
3,가슴 큰 태희,멜로/로맨스,2020,2.27,8.67,3,곧 결혼을 앞둔 예비 신혼 부부 차욱과 민주. 차욱은 직장 후배인 용우네 부부에게 ...,정보없음,정보없음,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200211_88/15...
4,개인교습,멜로/로맨스,2020,3.02,10.00,1,부잣집 프랑스 아가씨 마고에게 첫눈에 반한 발리의 가난한 어부 에카는 그녀에게 피아...,하리 산티카|도르카스 카핀,정보없음,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200226_62/15...


In [219]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11950 entries, 0 to 11949
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       11950 non-null  object 
 1   genre       11950 non-null  object 
 2   year        11950 non-null  int64  
 3   date        11950 non-null  float64
 4   rating      11950 non-null  float64
 5   vote_count  11950 non-null  int64  
 6   plot        11950 non-null  object 
 7   main_act    11950 non-null  object 
 8   supp_act    11950 non-null  object 
 9   page_url    11950 non-null  object 
 10  img_url     11950 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 1.0+ MB


In [220]:
df.rename(columns={"plot": "text"}, inplace=True)
df.head(5)

,title,genre,year,date,rating,vote_count,text,main_act,supp_act,page_url,img_url
0,1917,드라마|전쟁,2020,2.19,8.88,5017,제1차 세계대전이 한창인 1917년. 독일군에 의해 모든 통신망이 파괴된 상황 속에...,조지 맥케이|딘-찰스 채프먼,콜린 퍼스|베네딕트 컴버배치|마크 스트롱|앤드류 스캇|리차드 매든|에드리언 스카보로...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200212_161/1...
1,21 브릿지: 테러 셧다운,범죄|드라마|스릴러,2020,1.01,7.84,579,"뉴욕 맨해튼 중심에서 벌어진 경찰 연쇄 살해 사건, 범인을 잡기 위해 베테랑 경찰 ...",채드윅 보스만|J.K. 시몬스|시에나 밀러|테일러 키취,스테판 제임스|빅토리아 카타게나|키스 데이빗|샤이나 라이언|알렉산더 시디그|모로코 ...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20191227_205/1...
2,n번째 이별중,코미디|멜로/로맨스,2020,4.01,6.48,177,여자친구 ‘데비’에게 대차게 차여 충격받은 물리학 천재 ‘스틸먼’. 사랑하는 그녀의...,에이사 버터필드|소피 터너|스카일러 거손도,오브리 레이놀즈|윌 펠츠|질리안 조이|케이든 J. 그레고리,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200409_56/15...
3,가슴 큰 태희,멜로/로맨스,2020,2.27,8.67,3,곧 결혼을 앞둔 예비 신혼 부부 차욱과 민주. 차욱은 직장 후배인 용우네 부부에게 ...,정보없음,정보없음,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200211_88/15...
4,개인교습,멜로/로맨스,2020,3.02,10.00,1,부잣집 프랑스 아가씨 마고에게 첫눈에 반한 발리의 가난한 어부 에카는 그녀에게 피아...,하리 산티카|도르카스 카핀,정보없음,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200226_62/15...


### OpenSearch 도메인에 연결


In [221]:
# Add system path for utils
import sys

sys.path.insert(0, "/Users/jinhwan/Repository/Labs/opensearch-with-bedrock-workshop-kr/utils")

In [222]:
import boto3, json
from utils import get_cfn_outputs

region_name = "us-east-1"

cfn = boto3.client("cloudformation", region_name)
kms = boto3.client("secretsmanager", region_name)

stackname = "opensearch-workshop"
cfn_outputs = get_cfn_outputs(stackname, cfn)

aos_credentials = json.loads(
    kms.get_secret_value(SecretId=cfn_outputs["OpenSearchSecret"])["SecretString"]
)

aos_host = cfn_outputs["OpenSearchDomainEndpoint"]

In [223]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

auth = (aos_credentials["username"], aos_credentials["password"])

aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)

## 데이터 임베딩을 위한 Ingest Pipeline 생성


모델 아이디 설정


In [224]:
model_id = "jK-l048BhWoFUAg8WOdO"

인제스트 파이프라인 생성


In [17]:
pipeline = {
    "description": "An neural search pipeline for hotel index - OpenSearch-cohere-060124084807",
    "processors": [
        {
            "text_embedding": {
                "model_id": model_id,
                "field_map": {
                    "text": "vector_field",
                },
            }
        }
    ],
}

pipeline_id = "webtoon_embedding_pipeline"
aos_client.ingest.put_pipeline(id=pipeline_id, body=pipeline)

{'acknowledged': True}

인덱스 생성


In [229]:
index_name = "movie_semantic"

movie_semantic = {
    "settings": {
        "max_result_window": 15000,
        "analysis": {"analyzer": {"analysis-nori": {"type": "nori", "stopwords": "_korean_"}}},
        "index.knn": True,
        "default_pipeline": pipeline_id,
        "index.knn.space_type": "cosinesimil",
    },
    "mappings": {
        "properties": {
            "date": {
                "type": "float",
            },
            "genre": {
                "type": "text",
            },
            "img_url": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "main_act": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "page_url": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "text": {
                "type": "text",
            },
            "rating": {"type": "float"},
            "supp_act": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "title": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "vote_count": {"type": "long"},
            "year": {"type": "long"},
            "vector_field": {
                "type": "knn_vector",
                "dimension": 1024,
                "method": {"name": "hnsw", "space_type": "l2", "engine": "faiss"},
                "store": True,
            },
        }
    },
}

aos_client.indices.create(index=index_name, body=movie_semantic)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'movie_semantic'}

데이터 인제스트


In [230]:
from tqdm import tqdm
from opensearchpy import helpers

json_data = df.to_json(orient="records", lines=True)
docs = json_data.split("\n")[:-1]  # To remove the last empty line


def _generate_data():
    for doc in docs:
        yield {"_index": index_name, "_source": doc}


succeeded = []
failed = []
for success, item in helpers.parallel_bulk(
    aos_client, actions=_generate_data(), chunk_size=20, thread_count=2, queue_size=2
):
    if success:
        succeeded.append(item)
    else:
        failed.append(item)

BulkIndexError: ('1 document(s) failed to index.', [{'index': {'_index': 'movie_semantic', '_id': None, 'status': 400, 'error': {'type': 'status_exception', 'reason': 'Error from remote service: {"message":"Input is too long for requested model."}'}, 'data': '{"title":"416 \\ud504\\ub85c\\uc81d\\ud2b8 \\u201c\\ub9dd\\uac01\\uacfc \\uae30\\uc5b5\\u201d part1","genre":"\\ub2e4\\ud050\\uba58\\ud130\\ub9ac","year":2017,"date":4.15,"rating":9.5,"vote_count":2,"text":"PROJECT 1 <\\uc778\\uc591> 9\\uba85 \\ud76c\\uc0dd\\uc790\\uc758 \\uc218\\uc2b5\\uacfc \\ucc38\\uc0ac\\uc758 \\uc9c4\\uc0c1\\uaddc\\uba85\\uc744 \\uc704\\ud574 \\uc2dc\\uae09\\ud55c \\ubb38\\uc81c\\uac00 \\ubc14\\ub85c \\uc138\\uc6d4\\ud638\\uc758 \\uc628\\uc804\\ud55c \\uc778\\uc591\\uc774\\ub2e4. \\uc9d5\\ubd80\\ub294 \\uc778\\uc591\\uacfc\\uc815\\uc5d0 \\uc720\\uac00\\uc871\\uc758 \\ucc38\\uc5ec\\ub97c \\uc57d\\uc18d\\ud588\\uc9c0\\ub9cc \\uc5ec\\ub7ec \\uc774\\uc720\\ub97c \\ub300\\uac00\\uba70 \\uc720\\uac00\\uc871\\uc744 

잘 저장되었는지 확인


In [231]:
# Refresh the index to make the changes visible
aos_client.indices.refresh(index=index_name)

count = aos_client.count(index=index_name)
print(count)

{'count': 11224, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


검색 비교


In [324]:
def keyword_search(query_text):
    query = {
        "size": 10,
        "query": {
            "multi_match": {
                "query": query_text,
                "fields": ["text"],
            }
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_id"],
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["text"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_id", "_score", "title", "plot", "genre", "rating"]
    )
    display(query_result_df)

In [351]:
query_text = "슈퍼맨"

In [352]:
keyword_search(query_text)

,_id,_score,title,plot,genre,rating
0,_7AQ1I8BhWoFUAg8YGWm,8.672848,레고 무비,"배트맨, 슈퍼맨, 원더우먼, 인어공주, 초록닌자,198 몇년 우주인, 미켈란젤로, ...",애니메이션|액션|코미디,8.46
1,u7AQ1I8BhWoFUAg8rmyZ,8.142217,히어로,혼자 몸으로 애 키우느라 정신 없는 아빠 주연(오정세)과 어린이 드라마 ‘썬더맨’의...,드라마|코미디,6.16
2,vbAQ1I8BhWoFUAg8OmKO,8.073824,베카스,"이라크의 길거리에 사는 고아 형제 다나와 자나는 매일같이 티격태격 싸우지만, 서로 ...",드라마,8.43
3,XLAQ1I8BhWoFUAg8E18b,7.861473,배트맨 대 슈퍼맨: 저스티스의 시작,슈퍼맨과 조드 장군의 격렬한 전투 이후 메트로폴리스는 파괴되었고 슈퍼맨은 세계 최고...,액션|모험|판타지|SF,6.86
4,l7AQ1I8BhWoFUAg883L0,6.641743,여의도,"정계, 언론계, 증권계가 밀집해있는 황량한 여의도. 한 증권사에 근무하고 있는 황우...",드라마,6.23
5,R7AR1I8BhWoFUAg8JHYS,6.317495,핸콕,"X맨, 슈퍼맨, 배트맨, 스파이더맨이 가지고 있던 모든 능력에 누구도 건드릴 수 없...",액션|판타지|코미디|드라마,7.35
6,27AR1I8BhWoFUAg8N3e2,5.807272,할리우드랜드,"1959년 6월 16일, 로스앤젤레스. 인기 TV시리즈 <슈퍼맨의 모험>에서 슈퍼맨...",범죄|드라마|미스터리|스릴러,6.06
7,FbAR1I8BhWoFUAg8j36t,5.532388,스몰빌,"<슈퍼맨>의 주인공 클라크 켄트의 청년 시절을 그린 시리즈물. 12년전, 캔사스의 ...",SF|드라마,9.17
8,xbAR1I8BhWoFUAg8KXbT,4.976894,로빈슨 가족,"별난 세상, 별난 가족들! 디즈니만이 상상할 수 있는 미래로 초대합니다. 슈퍼맨, ...",애니메이션|모험|코미디|SF|가족,8.80
9,arAR1I8BhWoFUAg8THkV,4.976894,B형 남자친구,"운명 같은 사랑을 믿는 여대생 하미는, 우연히 길거리에서 마주친 영빈에게 첫눈에 필...",멜로/로맨스|코미디,4.88


In [353]:
def semantic_search(query_text):
    query = {
        "size": 10,
        "_source": {"excludes": ["vector_field"]},
        "query": {
            "neural": {"vector_field": {"query_text": query_text, "model_id": model_id, "k": 30}},
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_id"],
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["text"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_id", "_score", "title", "plot", "genre", "rating"]
    )
    display(query_result_df)

In [354]:
semantic_search(query_text)

,_id,_score,title,plot,genre,rating
0,TrAP1I8BhWoFUAg8w1hN,0.626057,몬몬몬 몬스터,인간을 해치는 괴물 괴물을 납치한 인간 친구를 괴롭힌 집단 모든걸 지켜본 당신 누가...,공포|스릴러,7.43
1,B7AP1I8BhWoFUAg8nVX0,0.614796,라스트 미션,평생 가족에게 잘못만 저질렀던 가장의 후회 자신의 잘못을 되돌리기 위한 마지막 선택...,드라마,8.81
2,4rAR1I8BhWoFUAg8U3ko,0.600135,생활의 달인,수십년간 한분야에 종사하며 부단한 열정과 노력으로 달인의 경지에 이르게된 사람들의 ...,다큐멘터리,6.25
3,N7AP1I8BhWoFUAg8zVl_,0.599203,어벤져스: 인피니티 워,"새로운 조합을 이룬 어벤져스, 역대 최강 빌런 타노스에 맞서 세계의 운명이 걸린 인...",액션|모험|판타지|SF,8.96
4,-7AQ1I8BhWoFUAg8vm0f,0.596628,사랑비,7 년대 시절의 순수했던 사랑의 정서와 2 12년 오늘날의 트렌디한 사랑법을 동시에...,드라마,8.91
5,-rAP1I8BhWoFUAg8tFaN,0.593570,틴 스피릿,"최고만이 설 수 있는 꿈의 무대, 춤, 노래, 그리고 열정 가득 세상을 빛낼 단 한...",드라마,6.34
6,wbAQ1I8BhWoFUAg8r2zY,0.591670,21세기 가족,2 12년 대한민국 현재를 살아가고 있는 가족들의 리얼한 이야기를 시크하고 시니컬한...,드라마,8.00
7,NrAP1I8BhWoFUAg82Fp_,0.591049,프로젝트 메자,평범한 고등학생이 어느날 갑자기 특별한 능력을 얻게되고 능력과 함께 자신을 쫓는 악...,액션|SF,4.43
8,OrAQ1I8BhWoFUAg8hmm3,0.587054,감자별 2013QR3,2 13년 어느 날 지구로 날아온 의문의 행성 '감자별' 때문에 벌어지는 노씨 일가...,드라마,9.49
9,0bAQ1I8BhWoFUAg8XmVm,0.586449,드라마 스페셜 - 꿈꾸는 남자,꿈을 통해 미래의 죽음을 예견하는 특별한 능력을 가진 남자가 한 여자를 구하기 위해...,드라마,8.00


## Clean Up


In [253]:
# aos_client.indices.delete(index=index_name)

{'acknowledged': True}